In [1]:
import tifffile
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import os
import UTXCTregister as reg
import openpyxl
from pathlib import Path

In [2]:
def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.
    
    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if (f.endswith('.tiff') or f.endswith('.tif'))])

    tiff_sequence = tifffile.TiffSequence(tiff_files)
    
    # Get the total number of TIFF files
    total_files = len(tiff_files)
    
    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)
            
            # Update progress
            pbar.update(1)
    
    return np.array(volume)

def write_sequence(folder_path, name, volume):
    """
    Save a 3D volume as a sequence of TIFF files in a folder.
    
    Args:
    folder_path (str): Path to the folder where TIFF files will be saved.
    name (str): Name of the TIFF files.
    volume (numpy.ndarray): A 3D array where each slice corresponds to an image.
    """

    folder_path = folder_path / name

    # Create the folder if it doesn't exist
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    # Save each slice as a TIFF file with progress bar
    with tqdm(total=volume.shape[0], desc="Saving") as pbar:
        for i in range(volume.shape[0]):
            tifffile.imwrite(f"{folder_path}/{name}_{i:04d}.tif", volume[i])
            pbar.update(1)
    
    print("Saving complete.")

In [3]:
database = pd.read_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv')

database = database.replace({np.nan: None})

In [4]:
database.columns

Index(['Id', 'Original', 'Eq', 'Frontal 90 Right', 'XCT padded', 'Binary',
       'Material Mask', 'Onlypores', 'UT', 'UT aligned', 'UT aligned amp',
       'UT registered', 'UT lente', 'UT aligned lente', 'UT aligned amp lente',
       'UT registered lente', 'Dataset PatchvsVolfrac 3x3',
       'Dataset PatchvsPacth 3x3', 'Dataset PatchvsVolfrac lente 3x3',
       'Dataset PatchvsPacth lente 3x3', 'Dataset PatchvsVolfrac 5x5',
       'Dataset PatchvsPacth 5x5', 'Dataset PatchvsVolfrac lente 5x5',
       'Dataset PatchvsPacth lente 5x5', 'Dataset PatchvsVolfrac 7x7',
       'Dataset PatchvsPacth 7x7', 'Dataset PatchvsVolfrac lente 7x7',
       'Dataset PatchvsPacth lente 7x7'],
      dtype='object')

In [5]:
ids = database['Id'].values

frontals  = database['Frontal 90 Right'].values

xct_paddeds = database['XCT padded'].values

amps = database['UT aligned amp'].values

rfs = database['UT aligned'].values

amps_lente = database['UT aligned amp lente'].values

rfs_lente = database['UT aligned lente'].values

registered_uts = database['UT registered'].values

registered_uts_lente = database['UT registered lente'].values

In [9]:
for i in range(len(ids)):

    #utvsxct register
    if registered_uts[i] == None: #if is not already registered

        try:

            print(f'Registering {ids[i]}')

            ut_file = str(rfs[i])

            ut_output_file = Path(ut_file[:-4] + '_registered.tif')

            ut = tifffile.imread(amps[i])
            ut = np.swapaxes(ut, 0, 1)
            ut = np.swapaxes(ut, 1, 2)

            rf = tifffile.imread(rfs[i])
            rf = np.swapaxes(rf, 0, 1)
            rf = np.swapaxes(rf, 1, 2)

            if xct_paddeds[i] == None: #if is not already padded, load the original xct

                xct_folder = Path(frontals[i])

                xct_output_folder = xct_folder.parent

                xct = tifffile.imread(frontals[i])
                xct = np.swapaxes(xct, 0, 1)
                xct = np.swapaxes(xct, 1, 2)
            
            else: #if padded, load the padded xct

                xct_folder = Path(xct_paddeds[i])

                xct_output_folder = xct_folder.parent

                xct = read_sequence(xct_paddeds[i])
                xct = np.swapaxes(xct, 0, 1)
                xct = np.swapaxes(xct, 1, 2)

            #check if xct is padded or not
            if xct_paddeds[i] == None: #It is not padded

                registered,xct_padded = reg.main(ut,rf,xct,False)

                tifffile.imwrite(ut_output_file, registered)

                xct_padded = np.swapaxes(xct_padded, 1, 2)
                xct_padded = np.swapaxes(xct_padded, 0, 1)

                write_sequence(xct_output_folder, 'padded', xct_padded)

                registered_uts[i] = str(ut_output_file)

                xct_paddeds[i] = str(xct_output_folder / 'padded')
            
            else:

                registered = reg.main(ut,rf,xct,True)

                tifffile.imwrite(ut_output_file, registered)

                registered_uts[i] = str(ut_output_file)

        
        except Exception as e:
            print(f'Error processing {ids[i]}')
            print(e)
    
    if registered_uts_lente[i] == None:

        try:

            print(f'Registering {ids[i]} lens')

            ut_file = str(rfs_lente[i])

            ut_output_file = Path(ut_file[:-4] + '_registered.tif')

            ut = tifffile.imread(amps_lente[i])
            ut = np.swapaxes(ut, 0, 1)
            ut = np.swapaxes(ut, 1, 2)

            rf = tifffile.imread(rfs_lente[i])
            rf = np.swapaxes(rf, 0, 1)
            rf = np.swapaxes(rf, 1, 2)

            if xct_paddeds[i] == None: #if is not already padded, load the original xct

                xct_folder = Path(frontals[i])

                xct_output_folder = xct_folder.parent

                xct = tifffile.imread(frontals[i])
                xct = np.swapaxes(xct, 0, 1)
                xct = np.swapaxes(xct, 1, 2)
            
            else: #if padded, load the padded xct

                xct_folder = Path(xct_paddeds[i])

                xct_output_folder = xct_folder.parent

                xct = read_sequence(xct_paddeds[i])
                xct = np.swapaxes(xct, 0, 1)
                xct = np.swapaxes(xct, 1, 2)

            #check if xct is padded or not
            if xct_paddeds[i] == None: #It is not padded

                registered,xct_padded = reg.main(ut,rf,xct,False)

                tifffile.imwrite(ut_output_file, registered)

                xct_padded = np.swapaxes(xct_padded, 1, 2)
                xct_padded = np.swapaxes(xct_padded, 0, 1)

                write_sequence(xct_output_folder, 'padded', xct_padded)

                registered_uts_lente[i] = str(ut_output_file)

                xct_paddeds[i] = str(xct_output_folder / 'padded')
            
            else:

                registered = reg.main(ut,rf,xct,True)

                tifffile.imwrite(ut_output_file, registered)

                registered_uts_lente[i] = str(ut_output_file)
        
        except Exception as e:
            print(f'Error processing {ids[i]}')
            print(e)

Registering 1_26


Progress: 100%|██████████| 350/350 [00:37<00:00,  9.26it/s]


Preprocessing
Not enough UT points
Error processing 1_26
Not enough UT points
Registering 1_10
Preprocessing
Preprocessed
Registering
Registered
Applying transformation
Transformation applied


Saving: 100%|██████████| 242/242 [01:19<00:00,  3.04it/s]


Saving complete.
Registering 1_10 lens


...

Preprocessing
Preprocessed
Registering
Registered
Applying transformation
Transformation applied
Registering 1_13
Preprocessing
Preprocessed
Registering
Registered
Applying transformation
Transformation applied


Saving: 100%|██████████| 249/249 [01:31<00:00,  2.72it/s]


Saving complete.
Registering 1_13 lens


Progress: 100%|██████████| 249/249 [00:35<00:00,  7.08it/s]


Preprocessing
Preprocessed
Registering
Registered
Applying transformation
Transformation applied


In [10]:
#update database

database['UT registered'] = registered_uts

database['UT registered lente'] = registered_uts_lente

database['XCT padded'] = xct_paddeds

In [11]:
database.to_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv', index=False)